## Microtask 1 ##
Produce a listing of the number of new committers per month, and the number of commits for each of them, as a table and as a CSV file. Use the GrimoireLab enriched index for git.

In [12]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, aggs
from pprint import pprint
from datetime import datetime
import pandas as pd

**Step1**: Instantiate the Elasticsearch client

In [14]:
es_client = Elasticsearch('http://localhost:9200')

**Step2**: Use the `p2o.py` script to fetch the git data for perceval repository and store the raw records in `git_raw` index, and the enriched data in the `git` index

In [16]:
%%bash
p2o.py --enrich --index git_raw --index-enrich git -e http://localhost:9200 --no_inc --debug git https://github.com/grimoirelab/perceval.git

2018-03-13 11:18:33,396 Debug mode activated
2018-03-13 11:18:33,398 Feeding Ocean from git (https://github.com/grimoirelab/perceval.git)
2018-03-13 11:18:33,418 Found version of ES instance at http://localhost:9200: 6.
2018-03-13 11:18:33,462 Creating OceanConf index http://localhost:9200/conf
2018-03-13 11:18:33,463 http://localhost:9200/git_raw/_search 
        { "size": 0, 
                "query" : {
                    "term" : { "origin" : "https://github.com/grimoirelab/perceval.git"  }
                 },
              
            "aggs": {
                "1": {
                  "max": {
                    "field": "metadata__updated_on"
                  }
                }
            }
        
        } 
2018-03-13 11:18:33,475 Incremental from: 2018-03-13 05:24:02+00:00
2018-03-13 11:18:36,314 Adding items to Ocean for <grimoire_elk.ocean.git.GitOcean object at 0x7f97847b5710> (1 items)
2018-03-13 11:18:36,314 Adding items to http://localhost:9200/git_raw/items/_bulk 

**Step3**: Create a DSL search object to query the enriched `git` index created earlier. The search query will bucket the commits data by the `author` field.<br> For each bucket, it will calculate two values:
1. The first commit by selecting the record with the minimum value on the field *'author_date'*, stored in the attribute `first_commit`
2. The number of commits for that author by calculating the number of unique values contained in the commit *'hash'* field, stored in the attribute `total_commits`

In [18]:
search = Search(using=es_client, index='git')
search.aggs.bucket('author', 'terms', field='author_name', size=100)\
           .metric('first_commit', 'min', field='author_date')\
           .metric('total_commits', 'cardinality', field='hash')
results = search.execute()

**Step4**: From the search results, create a dataframe holding the `first_commit`, `author` and `total_commit` values for each author

In [20]:
commiters = pd.DataFrame.from_records([ 
                  {
                    'first_commit' : datetime.utcfromtimestamp(bucket['first_commit']['value']/1000),
                    'author' : bucket['key'],
                    'total_commits' : bucket['total_commits']['value']
                  } 
                  for bucket in results['aggregations']['author']['buckets']
                ])

**Step5**: Sort the dataframe by the `total_commits` and produce a listing of commits for each author

In [22]:
commiters.sort_values(by='total_commits')
pprint (commiters[['author', 'total_commits']])

                            author  total_commits
0                  Santiago Dueñas            724
1                Valerio Cosentino            260
2                   Alberto Martín             51
3              Alvaro del Castillo             46
4       Jesus M. Gonzalez-Barahona             18
5                valerio cosentino              6
6                             quan              5
7           Miguel Ángel Fernández              3
8                         camillem              2
9                          valerio              2
10                     David Esler              1
11            David Pose Fernández              1
12                  Israel Herraiz              1
13  J. Manrique Lopez de la Fuente              1
14                 Luis Cañas Díaz              1
15                   Stephan Barth              1
16                           david              1


**Step6**: Group the `first_commit` column of the dataframe on a monthly basis, and obtain the counts for each grouping

In [23]:
commiters_by_month = commiters['first_commit']\
                     .groupby(commiters.first_commit.dt.to_period('M'))\
                     .agg('count')
pprint (commiters_by_month)

first_commit
2015-08    1
2015-12    2
2016-02    1
2016-03    2
2016-04    1
2016-09    1
2017-01    1
2017-09    2
2017-10    2
2017-11    1
2017-12    1
2018-01    1
2018-02    1
Freq: M, Name: first_commit, dtype: int64


**Step7**: Export the listings produced above in csv format

In [24]:
commiters.to_csv('author_commits.csv', columns=['author', 'total_commits'], index=False)
commiters_by_month.to_csv('commiters_by_month.csv')